# Multicriteria Disaster Response Problem

## Data Reader

### Libraries

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Reading the Data

In [86]:
directory = "/Users/yasemin/Desktop/sample_disaster_data.xlsx"
#directory = "/Users/yasemin/Desktop/avcilar_final_data.xlsx"

In [87]:
dataframe = pd.read_excel(open(directory, 'rb'), 
                   sheet_name='Part of Avcilar', 
                   engine='openpyxl')[:121] # for eliminating unnecessary parts

In [88]:
dataframe.head()

,inside,out,B,supply_t0,supply_t1,supply_t2,supply_t3,supply_t4,supply_t5,supply_t6,demand_t0,demand_t1,demand_t2,demand_t3,demand_t4,demand_t5,demand_t6,cost,capacity,blocked,rest_demand_t0,rest_demand_t1,rest_demand_t2,rest_demand_t3,rest_demand_t4,rest_demand_t5,rest_demand_t6,road_restoration_supply
0,1,1,1,150,152,303,455,606,758,909,0,0,0,0,0,0.0,0,999999999,0,0,23,90,29,29,20,60,89,500.0
1,1,2,0,200,202,404,606,808,1010,1212,0,0,0,0,0,0.0,0,5,50,1,70,13,29,8,83,94,78,NaN
2,1,3,0,250,253,505,758,1010,1263,1515,0,0,0,0,0,0.0,0,20,50,1,54,38,88,76,90,70,96,NaN
3,2,1,0,0,0,0,0,0,0,0,150,217,233,236,237,238.0,238,1,50,1,41,33,80,8,27,36,75,NaN
4,2,2,1,0,0,0,0,0,0,0,100,200,230,236,237,238.0,239,999999999,0,0,80,30,25,97,33,16,37,NaN


### Initial definitions

In [89]:
period_including_supply_columns = [col for col in dataframe.columns if "supply_t" in col]
time_list = []
periods = []
for column in period_including_supply_columns:
    time_list.append(int(column[8:]))
    periods.append(column[7:])
    
T = max(time_list) # 7 days
K = 1 # 1 commodity
N = len(np.unique(list(dataframe.inside))) # total number of nodes

supplies = dataframe[(f"supply_{i}" for i in periods)].sum(axis=1)
demands = dataframe[(f"demand_{i}" for i in periods)].sum(axis=1)

In [90]:
total_values = pd.concat([dataframe.inside, supplies, demands],axis=1)
total_values.columns = ["inside","total_supply","total_demand"]

In [91]:
nS = np.unique(list(total_values[(total_values.total_supply > 0)].inside.astype(int))) # supply node set
nD = np.unique(list(total_values[(total_values.total_demand > 0)].inside.astype(int))) # demand node set
nN = np.unique(list(total_values.inside)) # total node set

nT = time_list.copy()
nK = [0]

In [92]:
edge_dict = {}
for row in dataframe.iterrows():
    edge_dict[int(float(str(int(float(str(row[1].inside)))) + str(int(float(str(row[1].out))))))] = [int(float(row[1].inside)), int(float(row[1].out))]

In [93]:
nE = list(edge_dict.keys())
B = list(dataframe.B)

### Supplies

In [94]:
Sikt = []
for i in range(T+1):
    Sikt.append([])
    for j in range(K):
        Sikt[i].append({})

In [95]:
for t in range(T+1):
    for k in range(K):
        for n in nN:
            Sikt[t][k][n] = int(sum(np.unique(dataframe.loc[dataframe.inside == n, f"supply_t{t}"])))

In [96]:
Sikt

[[{1: 600, 2: 0, 3: 0}],
 [{1: 607, 2: 0, 3: 0}],
 [{1: 1212, 2: 0, 3: 0}],
 [{1: 1819, 2: 0, 3: 0}],
 [{1: 2424, 2: 0, 3: 0}],
 [{1: 3031, 2: 0, 3: 0}],
 [{1: 3636, 2: 0, 3: 0}]]

### Demands

In [97]:
djkt = []
for i in range(T+1):
    djkt.append([])
    for j in range(K):
        djkt[i].append({})

In [98]:
for t in range(T+1):
    for k in range(K):
        for n in nN:
            djkt[t][k][n] = int(sum(np.unique(dataframe.loc[dataframe.inside == n, f"demand_t{t}"])))

In [99]:
djkt

[[{1: 0, 2: 300, 3: 296}],
 [{1: 0, 2: 586, 3: 565}],
 [{1: 0, 2: 686, 3: 681}],
 [{1: 0, 2: 470, 3: 471}],
 [{1: 0, 2: 237, 3: 237}],
 [{1: 0, 2: 475, 3: 712}],
 [{1: 0, 2: 477, 3: 477}]]

### Edge Capacities

In [100]:
uijt = []
for t in range(T+1):
    uijt.append({})

In [101]:
for t in range(T+1):
    for n in nD:
        
        dataframe["edge"] = dataframe["inside"].astype(int).astype(str)\
        + dataframe["out"].astype(int).astype(str)
        
        edges = list(edge_dict.keys()) #list(dataframe.loc[dataframe.inside == n, "edge"])
        
        for edge in edges:
            #if str(edge).find(str(n)[:-2]) == 0:
            uijt[t][edge] = int(dataframe.loc[dataframe.edge == str(edge), "capacity"].unique())

In [102]:
uijt

[{11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0},
 {11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0},
 {11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0},
 {11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0},
 {11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0},
 {11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0},
 {11: 0, 12: 50, 13: 50, 21: 50, 22: 0, 23: 50, 31: 50, 32: 50, 33: 0}]

In [103]:
dataframe.capacity.fillna(0, inplace=True)

### Costs

In [104]:
Cijkt = []
for i in range(T+1):
    Cijkt.append([])
    for j in range(K):
        Cijkt[i].append({})

In [105]:
for t in range(T+1):
    for n in nN:
        for edge in list(edge_dict.keys()):
            if str(edge).find(str(int(n))) == 0:
                Cijkt[t][0][edge] = int(dataframe.loc[dataframe.edge == str(edge), "cost"].unique())

### Blocked Information

In [106]:
blocked = []
for i in range(T+1):
    blocked.append({})

In [107]:
for t in range(T+1):
    for edge in list(edge_dict.keys()):
        blocked[t][edge] = int(np.unique(dataframe.loc[dataframe.edge == str(edge), "blocked"]))

In [108]:
blocked

[{11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0},
 {11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0},
 {11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0},
 {11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0},
 {11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0},
 {11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0},
 {11: 0, 12: 1, 13: 1, 21: 1, 22: 0, 23: 1, 31: 1, 32: 1, 33: 0}]

### Restoration supply

In [109]:
# Number of resources needed to restore blocked arc (i,j)
bt = []
for t in range(T+1):
    bt.append([])

In [110]:
# TODO: This will depend on the previous used supply
for t in range(T+1):
    bt[t] = int(np.unique(dataframe["road_restoration_supply"].fillna(0).sum()))

In [111]:
bt

[500, 500, 500, 500, 500, 500, 500]

### Restoration demand

In [112]:
aij = []
for i in range(T+1):
    aij.append([])
    for j in range(K):
        aij[i].append({})

In [113]:
for t in range(T+1):
    for k in range(K):
        for n in nN:
            aij[t][k][n] = int(sum((dataframe[f"rest_demand_t{t}"])))

In [114]:
aij

[[{1: 604, 2: 604, 3: 604}],
 [{1: 454, 2: 454, 3: 454}],
 [{1: 490, 2: 490, 3: 490}],
 [{1: 423, 2: 423, 3: 423}],
 [{1: 523, 2: 523, 3: 523}],
 [{1: 471, 2: 471, 3: 471}],
 [{1: 448, 2: 448, 3: 448}]]

In [115]:
print("Data is ready")

Data is ready


## Optimization Model

In [116]:
import math
from pyomo.environ import *
import warnings
from collections import Counter
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
opt = SolverFactory('glpk')

In [117]:
class Problem_Model:
    def __init__(self, nE, nK, nT, nD):

        # Model Object
        self.Model = ConcreteModel()

        # Model Parameters
        self.Model.X = Var(nE, nK, nT, within=NonNegativeReals)
        self.Model.D = Var(nD, nK, nT, bounds=(0, np.inf), within=NonNegativeReals)
        self.Model.H = Var(nD, nK, nT, bounds=(0, np.inf), within=NonNegativeReals)
        self.Model.Q = Var(nD, nK, nT, bounds=(0, np.inf), within=NonNegativeReals)
        self.Model.W = Var(nE, nT, bounds=(0, np.inf), within=NonNegativeReals)
        self.Model.Y = Var(nE, nT, within=Binary)
        self.Model.T2 = Var(nD, nD, nK, nT, bounds=(0, np.inf), within=NonNegativeReals)
        
    # Function for model variable initialization & variables for objectives
    def model_initialization(self, nT, nK, nE, nD, Cijkt):

        # For blocked edges (by definition) and some initial pre-defined values
        for t in nT:
            for k in nK:
                for n in nN:
                    self.Model.X[int(str(int(n))+str(int(n))), k, t] = 0

        for k in nK:
            for n in nD:
                self.Model.H[n, k, min(nT)] = 0

        for t in nT:
            for n in nN:
                self.Model.W[int(str(int(n))+str(int(n))), t] = 0

        for t in nT:
            for n in nN:
                self.Model.Y[int(str(int(n))+str(int(n))), t] = 0  
            
        # Min Cost Objective
        self.Model.obj_mincost = 0
        self.Model.obj_unsatisfied = 0
        for t in nT:
            for k in nK:
                for e in nE:
                    self.Model.obj_mincost += Cijkt[t][k][e] * self.Model.X[e, k, t]
                    
        return self.Model

In [118]:
def model_objectives(Model, nD, nT, nK, nN, Sikt, djkt, nS, uijt, edge_dict, aij, bt, alpha):
    
    # UNSATISFIED DEMAND
    cumsum = {}
    cumsum_dict = {}
    for t in nT:
        for k in nK:
            for d1 in nD:
                if djkt[t][k][d1] > 0:
                    cumsum[d1] = djkt[t][k][d1]

        cumsum_fin = cumsum.copy()
        cumsum_dict[t] = cumsum_fin

    satisfied_cumsum = {}
    satisfied_cumsum_dict = {}
    for t in nT:
        for k in nK:
            for d1 in nD:
                if djkt[t][k][d1] > 0:
                    satisfied_cumsum[d1] = Model.Q[d1, k, t]

        satisfied_cumsum_fin = satisfied_cumsum.copy()
        satisfied_cumsum_dict[t] = satisfied_cumsum_fin

    unsatisfied_percentage = 0
    cumsum_dict_v2 = {}

    for t in nT:
        for k in nK:
            for d in nD:
                if djkt[t][k][d] > 0:

                    counter = 0
                    if d in cumsum_dict_v2.keys():
                        cumsum_dict_v2[d] += cumsum_dict[t][d]
                        counter += 1
                    else:
                        cumsum_dict_v2[d] = cumsum_dict[t][d]
                        counter += 1

            for key in cumsum_dict_v2.keys():
                unsatisfied_percentage += (Model.H[key, k, t] / cumsum_dict_v2[key])

            unsatisfied_percentage = (unsatisfied_percentage * math.exp(alpha))
            Model.obj_unsatisfied += unsatisfied_percentage
            unsatisfied_percentage = 0

    return Model

In [119]:
def model_constraints(Model, nD, nT, nK, nN, Sikt, djkt, nS, uijt, edge_dict, aij, bt, alpha):

    into = 0
    out = 0
    for t in nT:
        for k in nK:
            for e in nS:  # for all supply nodes
                if Sikt[t][k][e] > 0:

                    for i in edge_dict.keys():
                        if str(i).find(str(e)) == 0:
                            into += Model.X[i, k, t]
                        if str(i).find(str(e)) > 0:
                            out += Model.X[i, k, t]
                    Model.constraints.add(into - out == Sikt[t][k][e])
                    into = 0
                    out = 0

    
    into = 0
    out = 0
    for t in nT:
        for k in nK:
            for e in nD:  # for all demand nodes
                if djkt[t][k][e] > 0:
                    for i in edge_dict.keys():
                        if str(i).find(str(e)) == 0:
                            out += Model.X[i, k, t]
                        if str(i).find(str(e)) > 0:
                            into += Model.X[i, k, t]
                    Model.constraints.add(into - out + Model.H[e, k, t] == Model.D[e, k, t])
                    into = 0
                    out = 0  
                        

    for t in nT:
        for k in nK:
            for e in nN:  # for all transition nodes ( neither supply nor demand )
                if e not in nD and e not in nS:
                    for i in list(edge_dict.keys()):
                        if str(i).find(str(e)) == 0:
                            out += Model.X[i, k, t]
                        if str(i).find(str(e)) > 0:
                            into += Model.X[i, k, t]
                    Model.constraints.add(into - out == 0)
                    into = 0
                    out = 0
                    

    for t in nT:
        for k in nK:
            for d in nD:
                if t == 0:
                    Model.constraints.add(Model.D[d, k, t] == djkt[t][k][d])
                    Model.constraints.add(Model.H[d, k, t] <= Model.D[d, k, t])
                    Model.constraints.add(Model.Q[d, k, t] == Model.D[d, k, t] - Model.H[d, k, t])
                else:
                    Model.constraints.add(Model.D[d, k, t] == Model.H[d, k, t - 1] + djkt[t][k][d])
                    Model.constraints.add(Model.H[d, k, t] <= Model.D[d, k, t])
                    Model.constraints.add(Model.Q[d, k, t] == Model.D[d, k, t] - Model.H[d, k, t])
     
                    
    for t in nT:
        for edge in list(edge_dict.keys()):
            if edge not in [11,22,33]:
                Model.constraints.add(sum(Model.X[int(edge), k, t] for k in nK) <= uijt[t][edge])
           
    
    for t in nT:
        for edge in list(edge_dict.keys()):
            if edge in [11,22,33]:
                Model.constraints.add(sum(Model.X[int(edge), k, t] for k in nK) <= uijt[t][edge] * Model.Y[int(edge), t])

    
    for t in nT:
        Model.constraints.add(
            sum(Model.W[int(str(o) + str(d)), t] for (o, d) in zip(nS, nD)) <= bt[t])


    for t in nT:
        nT2 = RangeSet(0, t)
        for edge in list(edge_dict.keys()):
            if edge in [11,22,33]:
                Model.constraints.add(sum(Model.W[edge, t2] for t2 in nT2) >= aij[t][0][int(str(edge)[:1])] * Model.Y[edge, t])

    
    for t in nT:
        for k in nK:
            for edge in list(edge_dict.keys()):
                if edge in [11,22,33]:
                    
                    if int(str(edge)[1:]) in nD:
                        Model.constraints.add(Model.D[int(str(edge)[1:]), k, t] >= 0)
                        Model.constraints.add(Model.H[int(str(edge)[1:]), k, t] >= 0)
                        Model.constraints.add(Model.X[edge, k, t] >= 0)
        
    for t in nT:
        for k in nK:
            for n in nD:
                Model.constraints.add(Model.Q[n, k, t] >= 0)

    return Model

### Unscaled Min Cost Results

In [120]:
min_cost_results = []
min_unsatisfied_results = []

In [121]:
Target_Model = Problem_Model(nE, nK, nT, nD)
Target_Model = Target_Model.model_initialization(nT, nK, nE, nD, Cijkt)
Target_Model.constraints = ConstraintList()

Target_Model = model_objectives(Target_Model, nD, nT, nK, nN, Sikt, 
                                djkt, nS, uijt, edge_dict, aij, bt, alpha=0.3)
Target_Model = model_constraints(Target_Model, nD, nT, nK, nN, 
                                 Sikt, djkt, nS, uijt, edge_dict, 
                                 aij, bt, alpha=0.3)

Target_Model.obj = Objective(expr=((Target_Model.obj_mincost + Target_Model.obj_unsatisfied * 0.001)), 
                             sense=1)

Model_solution = opt.solve(Target_Model)

In [122]:
print(Model_solution)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 210
  Number of variables: 154
  Number of nonzeros: 460
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.009371042251586914



In [84]:
min_cost_results.append(Target_Model.obj_mincost())
min_unsatisfied_results.append(Target_Model.obj_unsatisfied())

ERROR: evaluating object as numeric value: X[12,0,0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object X[12,0,0]


ValueError: No value for uninitialized NumericValue object X[12,0,0]

In [85]:
min_cost_results, min_unsatisfied_results

([], [])

In [41]:
def excel_writer(nT, nK, nS, djkt, Model):
    data = {}
    demandnodes = []
    for i in nT:
        for j in nK:
            for key, value in djkt[i][j].items():
                if value != 0:
                    if key not in demandnodes:
                        demandnodes.append(key)
    df = pd.DataFrame(data)
    df[""] = demandnodes

    alpha = 0.3

    for i in nT:
        for j in nK:

            djktlist = []
            Dlist = []
            percentage = []
            Hlist = []
            QList = []
            discount = []

            for key, value in djkt[i][j].items():
                if key in demandnodes:
                    djktlist.append(value)
            for (key, value), (key2, value2), (key3, value3) in zip(Model.D.get_values().items(), Model.H.get_values().items(), Model.Q.get_values().items()):
                if key[0] in demandnodes and key[0] == key2[0] and j == key[1] and i == key[2] and key[0] == key3[0]:
                    Dlist.append(value)  # demand obtained in this period
                    Hlist.append(value2)
                    QList.append(value3)
                    discount.append(math.exp((alpha)))

            df["djkt(C" + str(j + 1) + "T" + str(i + 1) + ")"] = djktlist
            df["Djkt(C" + str(j + 1) + "T" + str(i + 1) + ")"] = Dlist
            df["Hjkt(C" + str(j + 1) + "T" + str(i + 1) + ")"] = Hlist
            df["Satisfied demand" + "(C" + str(j + 1) + "T" + str(i + 1) + ")"] = QList

            for m in range(len(djktlist)):
                percentage.append((QList[m] / Dlist[m] * 100))

    df.set_index('', drop=True, inplace=True)
    
    return df

### Unscaled Min Unsatisfied Demand

In [42]:
Target_Model = Problem_Model(nE, nK, nT, nD)
Target_Model = Target_Model.model_initialization(nT, nK, nE, nD, Cijkt)
Target_Model.constraints = ConstraintList()
Target_Model = model_objectives(Target_Model, nD, nT, nK, nN, Sikt, 
                                djkt, nS, uijt, edge_dict, aij, bt, alpha=0.3)
Target_Model = model_constraints(Target_Model, nD, nT, nK, nN, 
                                 Sikt, djkt, nS, uijt, edge_dict, 
                                 aij, bt, alpha=0.3)
Target_Model.obj = Objective(expr=(Target_Model.obj_unsatisfied + Target_Model.obj_mincost * 0.00001), 
                             sense=1)
Model_solution = opt.solve(Target_Model)

In [43]:
print(Model_solution)


Problem: 
- Name: unknown
  Lower bound: 14.2187796584793
  Upper bound: 14.2187796584793
  Number of objectives: 1
  Number of constraints: 210
  Number of variables: 154
  Number of nonzeros: 460
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.006187915802001953
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [44]:
min_cost_result_df = excel_writer(nT, nK, nS, djkt, Target_Model)
writer = pd.ExcelWriter(f'output_obj_trials.xlsx', engine='xlsxwriter')
min_cost_result_df.to_excel(writer, sheet_name=f"Only Min Cost")

min_unsatisfied_result_df = excel_writer(nT, nK, nS, djkt, Target_Model)
min_unsatisfied_result_df.to_excel(writer, sheet_name=f"Only Min Unsatisfied Demand")
writer.close()

In [45]:
min_cost_results.append(Target_Model.obj_mincost())
min_unsatisfied_results.append(Target_Model.obj_unsatisfied())

In [46]:
min_cost_results, min_unsatisfied_results

([225.0, 900.0], [17.063929434183, 14.209779658479315])

### Scaled Problem

In [ ]:
min_cost_result_df_scaled = []
min_unsatisfied_result_df_scaled = []

In [ ]:
def scaling(min_cost_results,  min_unsatisfied_results):

    max1 = np.max(min_cost_results)
    min1 = np.min(min_cost_results)
    max4 = np.max(min_unsatisfied_results)
    min4 = np.min(min_unsatisfied_results)

    scaling_factor_mincost = 1 / (max1 - min1)
    scaling_factor_unsatisfied = 1 / (max4 - min4)

    return min1, min4, scaling_factor_mincost, scaling_factor_unsatisfied

In [ ]:
min1, min4, scaling_factor_mincost, scaling_factor_unsatisfied = scaling(min_cost_results, 
                                                                         min_unsatisfied_results)

In [ ]:
Target_Model = Problem_Model(nE, nK, nT, nD)
Target_Model = Target_Model.model_initialization(nT, nK, nE, nD, Cijkt)
Target_Model.constraints = ConstraintList()
Target_Model = model_objectives(Target_Model, nD, nT, nK, nN, Sikt, 
                                djkt, nS, uijt, edge_dict, aij, bt, alpha=0.3)
Target_Model = model_constraints(Target_Model, nD, nT, nK, nN, 
                                 Sikt, djkt, nS, uijt, edge_dict, 
                                 aij, bt, alpha=0.3)

Target_Model.obj = Objective(expr=(((Target_Model.obj_mincost - min1) * scaling_factor_mincost)),
                              sense=1)  # min cost

Model_solution = opt.solve(Target_Model)

In [ ]:
print(Model_solution)

In [ ]:
min_cost_result_df_scaled.append((Target_Model.obj_mincost() - min1) * scaling_factor_mincost)
min_unsatisfied_result_df_scaled.append((Target_Model.obj_unsatisfied() - min4) * scaling_factor_unsatisfied)

In [ ]:
Target_Model = Problem_Model(nE, nK, nT, nD)
Target_Model = Target_Model.model_initialization(nT, nK, nE, nD, Cijkt)
Target_Model.constraints = ConstraintList()
Target_Model = model_objectives(Target_Model, nD, nT, nK, nN, Sikt, 
                                djkt, nS, uijt, edge_dict, aij, bt, alpha=0.3)
Target_Model = model_constraints(Target_Model, nD, nT, nK, nN, 
                                 Sikt, djkt, nS, uijt, edge_dict, 
                                 aij, bt, alpha=0.3)

Target_Model.obj = Objective(expr=(((Target_Model.obj_mincost - min1) * 0.001 * scaling_factor_mincost)),
                              sense=1)  # min unsatisfied demand

Model_solution = opt.solve(Target_Model)

In [ ]:
print(Model_solution)

In [ ]:
min_cost_result_df_scaled.append((Target_Model.obj_mincost() - min1) * scaling_factor_mincost)
min_unsatisfied_result_df_scaled.append((Target_Model.obj_unsatisfied() - min4) * scaling_factor_unsatisfied)

In [ ]:
min_cost_result_df_scaled, min_unsatisfied_result_df_scaled

In [ ]:
min_cost_result_df_scaled = excel_writer(nT, nK, nS, djkt, Target_Model)
writer = pd.ExcelWriter(f'output_obj_trials_scaled.xlsx', engine='xlsxwriter')
min_cost_result_df_scaled.to_excel(writer, sheet_name=f"Only Min Cost SC")

min_unsatisfied_result_df_scaled = excel_writer(nT, nK, nS, djkt, Target_Model)
min_unsatisfied_result_df_scaled.to_excel(writer, sheet_name=f"Only Min Unsatisfied Demand SC")
writer.close()

In [ ]:
min_unsatisfied_result_df

### Questions and TODOs

1. Yolların şu an hepsi açık (random kapatabiliyorum). Hangisi kapanacak? 

2. Supply & Demand değerlerini integer istiyor model, ancak yuvarlayınca demandler son period'lara doğru hep aynı değerlere yuvarlanıyor (logarithmic increase). Bu yüzden veri son günlere gelince çok sabit kalıyor. Bunun böyle olması uygun mu?

3. Aynı veri seti çok küçük supply & demand değerleriyle çalışıyor (eski örnek) ancak sayılar büyüyünce çalışmıyor? Bu mümkün mü?

4. Geçen hafta bir süre bazı objektifleri kombine edince aynı sonucu aldım (şu an öyle bir durum yok) bu hangi durumlarda mümkün olabilir?

5. Bizim modelimizin ana objektifi min unsatisfied demand + min cost * 0.00001 mi olacak, yoksa çeşitli kombinasyonları mı deneyeceğiz farklı case'ler olarak?